In [1]:
# CaptchaTrain.py
from CaptchaGenerator import char_set
from CaptchaGenerator import gen_captcha_text_and_image, convert_image_to_gray, generate_captcha_text_label, \
   text_label_turn_to_char_list, pred_label_turn_to_char_list
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import math

C:\Users\WYG\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\WYG\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\WYG\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\WYG\AppData\Roaming\Python\Python37\site-packages

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 基本参数设置：验证码图像的大小、验证码字符串长度、训练批样本大小
IMAGE_WIDTH = 160
IMAGE_HEIGHT = 60
TEXT_LEN = 4
CHAR_SET = char_set
BATCH_SIZE = 64
VALIDATION_SIZE = 100
TEST_SIZE = 100
ITERATION = 5

if not os.path.exists("./tmp/"):
   os.mkdir("./tmp")
if not os.path.exists("./train_image/"):
   os.mkdir("./train_image/")
if not os.path.exists("./validation_image/"):
   os.mkdir("./validation_image/")
if not os.path.exists("./test_image/"):
   os.mkdir("./test_image/")


In [3]:
# 生成一个训练batch
def get_next_batch(batch_size, mode="train"):
   im_save_path = ""
   batch_x = np.zeros(shape=[batch_size, IMAGE_WIDTH * IMAGE_HEIGHT])
   batch_y = np.zeros(shape=[batch_size, TEXT_LEN * len(CHAR_SET)])
   for index in range(batch_size):
      tex, img = gen_captcha_text_and_image(IMAGE_WIDTH, IMAGE_HEIGHT, TEXT_LEN, CHAR_SET)
      im = Image.fromarray(img)
      if mode == "train":# 训练集
         im_save_path = "./train_image/" + str(index) + "_train.jpg"
      elif mode == "validation":# 验证集
         im_save_path = "./validation_image/" + str(index) + "_validation.jpg"
      elif mode == "test":# 测试集
         im_save_path = "./test_image/" + str(index) + "_test.jpg"
      im.save(im_save_path)
      gray_image, gray_image_matrix = convert_image_to_gray(img)
      batch_x[index, :] = gray_image_matrix
      batch_y[index, :] = generate_captcha_text_label(tex, CHAR_SET)

   return batch_x, batch_y

In [4]:
# 测试
# batch_x, batch_y = get_next_batch(BATCH_SIZE)
# print(batch_x[0], batch_y[0])

X = tf.placeholder(tf.float32, [None, IMAGE_HEIGHT * IMAGE_WIDTH])
Y = tf.placeholder(tf.float32, [None, TEXT_LEN * len(CHAR_SET)])
keep_prob = tf.placeholder(tf.float32)


# 定义生成w变量的函数
def weight_variable(shape):
   initial = tf.truncated_normal(shape, stddev=0.1)
   return tf.Variable(initial)


# 定义生成b变量的函数
def bias_variable(shape):
   initial = tf.constant(0.1, shape=shape)
   return tf.Variable(initial)


# 定义卷积函数,x是输入的图像，W是此卷积层的权重矩阵
def conv2d(x, w):
   return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME")


# 定义池化函数，x是输入的矩阵
def max_pool2x2(x):
   return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")



In [5]:
# 定义神经网络函数
def cnn(x_input, hei, wid, text_len, len_char_set):
   x = tf.reshape(x_input, shape=[-1, hei, wid, 1])
   # x：一个shape为(BATCH_SIZE,160,60,1)的张量
   # conv_1p输出：一个shape为(BATCH_SIZE,80,30,32)的张量
   # conv_2p输出：一个shape为(BATCH_SIZE,40,15,64)的张量
   # conv_3p输出：一个shape为(BATCH_SIZE,20,8,64)的张量
   # conv_3r输出：一个shape为(BATCH_SIZE,20X8X64)的张量
   # fc_1输出：一个shape为(BATCH_SIZE,1024)的张量
   # out输出：一个shape为(BATCH_SIZE,40)的张量
   conv_width_input = wid
   conv_height_input = hei
   w_c1 = weight_variable([3, 3, 1, 32])
   b_c1 = bias_variable([32])
   w_c2 = weight_variable([3, 3, 32, 64])
   b_c2 = bias_variable([64])
   w_c3 = weight_variable([3, 3, 64, 64])
   b_c3 = bias_variable([64])
   full_connect_width = math.ceil(conv_width_input / 8)
   full_connect_height = math.ceil(conv_height_input / 8)
   w_d1 = weight_variable([full_connect_width * full_connect_height * 64, 1024])
   b_d1 = bias_variable([1024])
   w_out = weight_variable([1024, text_len * len_char_set])
   b_out = bias_variable([text_len * len_char_set])

   conv_1c = tf.nn.relu(conv2d(x, w_c1) + b_c1)
   conv_1p = max_pool2x2(conv_1c)
   conv_1d = tf.nn.dropout(conv_1p, keep_prob)
   conv_2c = tf.nn.relu(conv2d(conv_1d, w_c2) + b_c2)
   conv_2p = max_pool2x2(conv_2c)
   conv_2d = tf.nn.dropout(conv_2p, keep_prob)
   conv_3c = tf.nn.relu(conv2d(conv_2d, w_c3) + b_c3)
   conv_3p = max_pool2x2(conv_3c)
   conv_3d = tf.nn.dropout(conv_3p, keep_prob)
   conv_3r = tf.reshape(conv_3d, [-1, full_connect_width * full_connect_height * 64])
   fc_1 = tf.nn.relu(tf.matmul(conv_3r, w_d1) + b_d1)
   fc_1d = tf.nn.dropout(fc_1, keep_prob)
   out = tf.matmul(fc_1d, w_out) + b_out

   return out


In [ ]:
# 将预测值规整成BATCH_SIZE个TEXT_LEN行len(CHAR_SET)列的矩阵，这样每行就对应字符序列中的一个字符的标签
output = cnn(X, IMAGE_WIDTH, IMAGE_HEIGHT, TEXT_LEN, len(CHAR_SET))
# 定义loss函数和优化算法
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=Y))
train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
# 预测值
predict = tf.argmax(tf.reshape(output, [-1, TEXT_LEN, len(CHAR_SET)]), 2)
# 将真实标签规整成BATCH_SIZE个TEXT_LEN行len(CHAR_SET)列的矩阵，这样每行就对应字符序列中的一个字符的标签
real_value = tf.argmax(tf.reshape(Y, [-1, TEXT_LEN, len(CHAR_SET)]), 2)
# 计算预测准确率
correct_pre = tf.equal(predict, real_value)
accuracy = tf.reduce_mean(tf.cast(correct_pre, tf.float32))

# 保存模型的对象saver
saver = tf.train.Saver()

with tf.Session() as sess_train:
   sess_train.run(tf.global_variables_initializer())
   if os.path.exists("./tmp/checkpoint"):
      # 判断模型是否存在，如果存在则从模型中恢复变量
      saver.restore(sess_train, tf.train.latest_checkpoint("./tmp/"))
   step = 0
   while True:
      batch_x_train, batch_y_train = get_next_batch(BATCH_SIZE)
      _, batch_loss = sess_train.run([train_op, loss],
                                     feed_dict={X: batch_x_train, Y: batch_y_train, keep_prob: 0.75})
      if step % 5 == 0:
         # 每训练5次打印一次loss值
         print("iteration：%d , batch_loss：%s" % (step, batch_loss))
      if step % 100 == 0:
         # 每训练100次保存一次模型
         saver.save(sess_train, "./tmp/train_model", global_step=step)
         # 每训练100次计算并打印一次准确率
         batch_x_validation, batch_y_validation = get_next_batch(VALIDATION_SIZE, mode="validation")
         acc = sess_train.run(accuracy, feed_dict={X: batch_x_validation, Y: batch_y_validation, keep_prob: 1})
         print("iteration：%d , acc：%s" % (step, acc))
         # 如果准确率大于设定值,保存模型,完成训练
         if acc > 0.9:
            saver.save(sess_train, "./tmp/train_model", global_step=step)
            break
      step = step + 1
# 建立用于测试模型的Session会话

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./tmp/train_model-1400
iteration：0 , batch_loss：0.3254881
iteration：0 , acc：0.0925
iteration：5 , batch_loss：0.32565576
iteration：10 , batch_loss：0.32693452
iteration：15 , batch_loss：0.32594782
iteration：20 , batch_loss：0.32663256
iteration：25 , batch_loss：0.32668543
iteration：30 , batch_loss：0.32595006
iteration：35 , batch_loss：0.32773232
iteration：40 , batch_loss：0.32714808
iteration：45 , batch_loss：0.3245657
iteration：50 , batch_loss：0.32604808
iteration：55 , batch_loss：0.3257455
iteration：60 , batch_loss：0.326481
iteration：65 , batch_loss：0.32604855
iteration：70 , batch_loss：0.32696822
iteration：75 , batch_loss：0.32764626
iteration：80 , batch_loss：0.3

iteration：875 , batch_loss：0.32630366
iteration：880 , batch_loss：0.3274804
iteration：885 , batch_loss：0.32793847
iteration：890 , batch_loss：0.32846212
iteration：895 , batch_loss：0.3281245
iteration：900 , batch_loss：0.32645908
iteration：900 , acc：0.0875
iteration：905 , batch_loss：0.32662702
iteration：910 , batch_loss：0.32622874
iteration：915 , batch_loss：0.32642317
iteration：920 , batch_loss：0.3254339
iteration：925 , batch_loss：0.32642785
iteration：930 , batch_loss：0.3270634
iteration：935 , batch_loss：0.32639492
iteration：940 , batch_loss：0.32585758
iteration：945 , batch_loss：0.32558793
iteration：950 , batch_loss：0.32570687
iteration：955 , batch_loss：0.32638878
iteration：960 , batch_loss：0.32488543
iteration：965 , batch_loss：0.32515883
iteration：970 , batch_loss：0.326144
iteration：975 , batch_loss：0.32736138
iteration：980 , batch_loss：0.32639226
iteration：985 , batch_loss：0.32719728
iteration：990 , batch_loss：0.325356
iteration：995 , batch_loss：0.32659176
iteration：1000 , batch_loss：0.3

iteration：1900 , acc：0.11
iteration：1905 , batch_loss：0.32637626
iteration：1910 , batch_loss：0.32562
iteration：1915 , batch_loss：0.32619825
iteration：1920 , batch_loss：0.326873
iteration：1925 , batch_loss：0.3260923
iteration：1930 , batch_loss：0.32458416
iteration：1935 , batch_loss：0.32419753
iteration：1940 , batch_loss：0.32716107
iteration：1945 , batch_loss：0.32723933
iteration：1950 , batch_loss：0.32459918
iteration：1955 , batch_loss：0.32482862
iteration：1960 , batch_loss：0.32682815
iteration：1965 , batch_loss：0.32643276
iteration：1970 , batch_loss：0.32703218
iteration：1975 , batch_loss：0.32596207
iteration：1980 , batch_loss：0.32622743
iteration：1985 , batch_loss：0.32633573
iteration：1990 , batch_loss：0.3264009
iteration：1995 , batch_loss：0.32419655
iteration：2000 , batch_loss：0.3244051
iteration：2000 , acc：0.0775
iteration：2005 , batch_loss：0.325827
iteration：2010 , batch_loss：0.3248014
iteration：2015 , batch_loss：0.32560974
iteration：2020 , batch_loss：0.32673582
iteration：2025 , batc

iteration：2930 , batch_loss：0.3255717
iteration：2935 , batch_loss：0.32575592
iteration：2940 , batch_loss：0.32394275
iteration：2945 , batch_loss：0.32641813
iteration：2950 , batch_loss：0.3255481
iteration：2955 , batch_loss：0.32692537
iteration：2960 , batch_loss：0.32575333
iteration：2965 , batch_loss：0.32457542
iteration：2970 , batch_loss：0.32581982
iteration：2975 , batch_loss：0.32504502
iteration：2980 , batch_loss：0.32563144
iteration：2985 , batch_loss：0.32738155
iteration：2990 , batch_loss：0.32658225
iteration：2995 , batch_loss：0.32464814
iteration：3000 , batch_loss：0.325537
iteration：3000 , acc：0.1025
iteration：3005 , batch_loss：0.3268074
iteration：3010 , batch_loss：0.32600254
iteration：3015 , batch_loss：0.3246504
iteration：3020 , batch_loss：0.32548395
iteration：3025 , batch_loss：0.3258376
iteration：3030 , batch_loss：0.32595608
iteration：3035 , batch_loss：0.32681113
iteration：3040 , batch_loss：0.32493824
iteration：3045 , batch_loss：0.32480302
iteration：3050 , batch_loss：0.32415062
iter

iteration：3955 , batch_loss：0.3258315
iteration：3960 , batch_loss：0.3269542
iteration：3965 , batch_loss：0.32695517
iteration：3970 , batch_loss：0.3257038
iteration：3975 , batch_loss：0.32560173
iteration：3980 , batch_loss：0.3260687
iteration：3985 , batch_loss：0.3243574
iteration：3990 , batch_loss：0.32556853
iteration：3995 , batch_loss：0.32612124
iteration：4000 , batch_loss：0.32649463
iteration：4000 , acc：0.105
iteration：4005 , batch_loss：0.32497263
iteration：4010 , batch_loss：0.3252012
iteration：4015 , batch_loss：0.32626295
iteration：4020 , batch_loss：0.32643205
iteration：4025 , batch_loss：0.32624623
iteration：4030 , batch_loss：0.32588702
iteration：4035 , batch_loss：0.32503116
iteration：4040 , batch_loss：0.3251632
iteration：4045 , batch_loss：0.32607085
iteration：4050 , batch_loss：0.3258992
iteration：4055 , batch_loss：0.3245302
iteration：4060 , batch_loss：0.3254283
iteration：4065 , batch_loss：0.32581204
iteration：4070 , batch_loss：0.32518977
iteration：4075 , batch_loss：0.32613835
iteratio

In [ ]:
with tf.Session() as sess_test:
   sess_test.run(tf.global_variables_initializer())
   saver.restore(sess_test, tf.train.latest_checkpoint("./tmp/"))
   for i in range(ITERATION):
      batch_x_test, batch_y_test = get_next_batch(TEST_SIZE, mode="test")
      pred, real, acc = sess_test.run([predict, real_value, accuracy],
                                      feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1})
      # 注意pred和real都是tensorflow的张量tensor，虽然它们和多维数组形式非常类似，但不是数组，不是可迭代对象
      # tensorflow的张量tensor不能直接用在for循环里迭代，另外数组中的元素如果是float型不能直接用作数组的下标
      pred_int = tf.cast(pred, tf.int32)
      real_int = tf.cast(real, tf.int32)
      pred_array = pred_int.eval(session=sess_test)
      real_array = real_int.eval(session=sess_test)
      # pred, real的张量形式是2维矩阵，TEXT_SIZE行X4列矩阵，之所以是4列是因为前面用tf.argmax逐行提取了每行最大值的下标，一共四行，所以是4个下标
      # 上面两步先将张量的值全部转为int型，然后将张量转换成多维数组
      # 把张量转换成多维数组后，我们就可以进行迭代，然后使用pred_label_turn_to_char_list函数将标签转换成对应的字符序列
      for j in range(TEST_SIZE):
         # 这里预测值的标签经过tf.argmax只提取出了四个字符的下标，真实值的标签也这么处理了
         # 因此由标签得到字符序列都使用函数pred_label_turn_to_char_list
         pred_char_list = pred_label_turn_to_char_list(pred_array[j], TEXT_LEN, CHAR_SET)
         real_char_list = pred_label_turn_to_char_list(real_array[j], TEXT_LEN, CHAR_SET)
         print("第{}轮ITERATION中第{}个验证码预测：预测验证码为：{} 真实验证码为：{}".format(i + 1, j + 1, pred_char_list, real_char_list))
      print("第{}轮ITERATION识别测试正确率：{}".format(i + 1, acc))